# Laboratory Exercise: Predicting Champion Picks in the LoL Worlds Final

**Goal:** In this laboratory exercise, you will build a machine learning model capable of predicting whether a League of Legends champion will be selected for the **Season 13 Worlds Final**.
Using detailed performance statistics from **Season 12** and **Season 13**, you will explore, clean, and merge the datasets, construct meaningful features, analyze champion trends, and ultimately train a classification model that identifies which champions are strong enough to appear on the world stage.


## 1. Overview of the Task

You are given two datasets:

- `season_12_data.csv`
- `season_13_data.csv`

Each dataset contains the following features:

- **Name**
  The name of the champion (e.g., *Ahri*, *Garen*, *Lee Sin*).

- **Class**
  The gameplay class or archetype of the champion.
  Possible values include: **Fighter**, **Assassin**, **Mage**, **Marksman**, **Support**, **Tank**.

- **Role**
  The primary lane or position where the champion is played.
  Possible roles: **Top**, **Mid**, **ADC**, **Support**, **Jungle**.

- **Tier**
  The overall performance/strength ranking of the champion for that season.
  Possible tiers: **S+**, **S**, **A**, **B**, **C**, **D**.

- **Trend**
  The performance trend or momentum indicator (e.g., positive/negative score trend across the season).

- **Win %**
  The win rate percentage of the champion in the given role and season (e.g., `51%`).

- **Role %**
  The percentage of games in which this champion is played in the **given role** (e.g., a champion being played top lane 80% of the time).

- **Pick %**
  The pick rate percentage, i.e., how often the champion is selected overall.

- **Ban %**
  The ban rate percentage.
  - This is split by side, e.g., `"Blue (23%) / Red (34%)"`, indicating separate ban rates depending on the team side.

- **KDA**
  The Kill/Death ratio of the champion for the season.

Your final objective is to build a machine learning model that predicts whether a champion will be selected for the **Season 13 Worlds Final** (fictionally played every two seasons).

To achieve this, you will build a complete data-science pipeline:
- Data loading
- Column renaming
- Full outer merging
- Data cleaning (strings, percentages, KDA formatting)
- Handling missing values
- Exploratory Data Analysis (EDA)
- Feature selection
- Categorical encoding
- Target variable creation
- Train–test splitting
- ML model training
- Final evaluation

## 2. Renaming Columns by Season

Both datasets share identical column names.
To avoid confusion after merging, rename all non-key columns so that their season is clearly indicated.

Keep the following **unchanged**:
- `Name`
- `Role`

Rename all other columns using this format:

- `Win % (season 12)`
- `Win % (season 13)`

This ensures clarity and prevents accidental overwriting during merging or preprocessing.

## 3. Full Outer Join on `Name` and `Role`

Perform a **full outer merge** between the two datasets using:

- `Name`
- `Role`

Why full outer join?:
- Some champions exist in Season 13 but not in Season 12 (new releases)
- Some champions exist in Season 12 but only in certain roles, resulting in partial feature availability

A full outer join ensures:
- Champions appearing only in Season 12 are included
- Champions appearing only in Season 13 are included
- Shared champions receive both seasons’ features
- Missing values naturally appear where data is unavailable

After merging:
- Inspect the shape
- Check for missing values
- Show initial descriptive statistics

How to perform the merge:

- `df_full = season12.merge(season13, on=["Name","Role"], how="outer")`

## 4. Cleaning Numeric-Like Columns Stored as Strings

Several columns appear numeric but are actually stored as strings. These must be converted into clean, consistent numerical values before any modeling or missing-value handling.

Examples of problematic formats:

- `"78%"` → should become `78.0`
- `"Blue (23%)/Red (34%)"` → Ban rate split by side

### Your tasks:

1. **Strip percentage symbols (`%`)** from all percentage columns.
2. **Handle the Ban % special case**.
   - The `Ban %` column contains values such as: `"Blue (23%)/Red (34%)"`
   - Take the **sum** of them: `sum(23, 34)`
3. **Convert all cleaned values to `float`**
4. Ensure that all missing values stay **`NaN`**

This preprocessing step must be completed **before handling missing values**

## 5. Handling Missing Values

You must:

- Analyze why values are missing
- Apply appropriate techniques:
  - **Simple Imputation (mean, mode, median)**
  - **Advanced Imputation (MICE, KNN)**
  - **Dropping rows/columns** when justified

Your decisions must be motivated.

## 6. Exploratory Data Analysis (EDA)

Perform EDA to understand data patterns, distributions, and relationships.

Suggested visualizations:

- Distribution plots (histograms)
- Boxplots for numeric columns
- Correlation heatmap (numeric features)
- Role distribution (Season 12 & 13)
- Tier distribution
- Missing-value visualization heatmap

Use visualizations to justify feature selection and preprocessing choices.

## 7. Feature Selection

Not all features are equally valuable.
You must determine:

- Which numeric features matter most
- Which categorical features are useful
- Whether any columns are redundant or irrelevant

## 8. Creating the Target Column: `World Cup Suitable`

Create a new binary label based on Tier performance:

### A champion is labeled **1 (World Cup Suitable)** if:

- `Tier (season 12)` is **S+**,
  **OR**
- `Tier (season 13)` is **S** or **S+**

Otherwise → label **0**.

This new column will be your **prediction target**.

After generating the label:

- Decide whether to drop or encode the Tier columns
- Ensure proper categorical encoding if kept

## 9. Encoding Categorical Features

Choose the encoding strategy:

- **One-Hot Encoding**
- **Ordinal Encoding**

## 10. Train–Test Split

Split your dataset into:

- **Training set** (80%)
- **Test set** (20%)

## 11. Model Training

Train at least one classification model

## 12. Model Evaluation

Evaluate your model using classification metrics

In [1]:
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder
import numpy as np
import missingno as msno
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

In [2]:
df_1 = pd.read_csv("season_12_data.csv")
df_2 = pd.read_csv("season_13_data.csv")

In [3]:
new_column_1 = []

for col in df_1.columns:
    if col == "Name" or col == "Role":
        new_column_1.append(col)
        continue
    new_column = f"{col} (season 12)"
    new_column_1.append(new_column)

new_column_2 = []

for col in df_2.columns:
    if col == "Name" or col == "Role":
        new_column_2.append(col)
        continue
    new_column1 = f"{col} (season 13)"
    new_column_2.append(new_column1)

In [4]:
df_1.columns = new_column_1
df_2.columns = new_column_2

In [5]:
df_full = df_1.merge(df_2, on=["Name", "Role"], how="outer")

In [6]:
df_full

,Name,Class (season 12),Role,Tier (season 12),Trend (season 12),Win % (season 12),Role % (season 12),Pick % (season 12),Ban % (season 12),KDA (season 12),Class (season 13),Tier (season 13),Trend (season 13),Win % (season 13),Role % (season 13),Pick % (season 13),Ban % (season 13),KDA (season 13)
0,Aatrox,Fighter,TOP,A,6.52,49.97%,94.62%,4.43%,Blue (0.83%)/Red (1.2%),1.97,Fighter,S,-31.86,47.68%,91.63%,6.62%,Blue (9.8%)/Red (2.18%),1.77
1,Ahri,Mage,MID,A,-0.24,49.93%,93.47%,4.62%,Blue (0.2%)/Red (0.84%),2.56,Mage,S,4.55,49.50%,94.65%,5.81%,Blue (1.54%)/Red (0.19%),2.58
2,Akali,Assassin,MID,S,6.51,48.59%,65.65%,8.16%,Blue (6.51%)/Red (6.37%),2.34,Assassin,S,4.33,48.41%,75.74%,8.11%,Blue (5.74%)/Red (7.28%),2.37
3,Akali,Assassin,TOP,A,3.34,48.57%,34.06%,4.24%,Blue (5.57%)/Red (7.31%),2.04,Assassin,C,-1.51,45.92%,23.50%,2.55%,Blue (0.47%)/Red (12.55%),2.00
4,Akshan,Marksman,MID,S,0.65,51.46%,58.01%,4.83%,Blue (5.07%)/Red (16.84%),2.23,Marksman,A,0.34,51.62%,66.03%,2.75%,Blue (0.06%)/Red (3.73%),2.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,Zilean,Support,MID,A,1.35,54.19%,14.12%,0.56%,Blue (0.7%)/Red (0.7%),3.46,Support,A,-3.47,53.33%,15.52%,0.49%,Blue (0.85%)/Red (0.55%),3.59
254,Zilean,Support,SUPPORT,A,1.68,50.83%,81.95%,3.38%,Blue (0.68%)/Red (0.72%),3.03,Support,A,-2.49,50.95%,79.72%,2.67%,Blue (1.12%)/Red (0.28%),3.11
255,Zoe,Mage,MID,A,4.62,49.84%,94.18%,3.62%,Blue (0.02%)/Red (1.39%),2.58,Mage,A,2.43,49.14%,87.85%,3.45%,Blue (0.27%)/Red (1.18%),2.60
256,Zoe,NaN,SUPPORT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mage,D,-3.55,47.73%,10.03%,0.37%,Blue (0.93%)/Red (0.52%),2.14


In [7]:
df_full.isnull().sum()

Name                   0
Class (season 12)     27
Role                   0
Tier (season 12)      26
Trend (season 12)     26
Win % (season 12)     26
Role % (season 12)    26
Pick % (season 12)    26
Ban % (season 12)     26
KDA (season 12)       26
Class (season 13)     14
Tier (season 13)      14
Trend (season 13)     14
Win % (season 13)     14
Role % (season 13)    14
Pick % (season 13)    14
Ban % (season 13)     14
KDA (season 13)       14
dtype: int64

In [8]:
df_full["Class (season 12)"].unique()

array(['Fighter', 'Mage', 'Assassin', 'Marksman', 'Tank', nan, 'Support'],
      dtype=object)

In [9]:
percentage_column = [col for col in df_full.columns if "%" in col and "Ban %" not in col]

In [10]:
for col in percentage_column:
    df_full[col] = df_full[col].apply(lambda x: x.replace("%","") if not pd.isna(x) else x)

df_full

,Name,Class (season 12),Role,Tier (season 12),Trend (season 12),Win % (season 12),Role % (season 12),Pick % (season 12),Ban % (season 12),KDA (season 12),Class (season 13),Tier (season 13),Trend (season 13),Win % (season 13),Role % (season 13),Pick % (season 13),Ban % (season 13),KDA (season 13)
0,Aatrox,Fighter,TOP,A,6.52,49.97,94.62,4.43,Blue (0.83%)/Red (1.2%),1.97,Fighter,S,-31.86,47.68,91.63,6.62,Blue (9.8%)/Red (2.18%),1.77
1,Ahri,Mage,MID,A,-0.24,49.93,93.47,4.62,Blue (0.2%)/Red (0.84%),2.56,Mage,S,4.55,49.50,94.65,5.81,Blue (1.54%)/Red (0.19%),2.58
2,Akali,Assassin,MID,S,6.51,48.59,65.65,8.16,Blue (6.51%)/Red (6.37%),2.34,Assassin,S,4.33,48.41,75.74,8.11,Blue (5.74%)/Red (7.28%),2.37
3,Akali,Assassin,TOP,A,3.34,48.57,34.06,4.24,Blue (5.57%)/Red (7.31%),2.04,Assassin,C,-1.51,45.92,23.50,2.55,Blue (0.47%)/Red (12.55%),2.00
4,Akshan,Marksman,MID,S,0.65,51.46,58.01,4.83,Blue (5.07%)/Red (16.84%),2.23,Marksman,A,0.34,51.62,66.03,2.75,Blue (0.06%)/Red (3.73%),2.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,Zilean,Support,MID,A,1.35,54.19,14.12,0.56,Blue (0.7%)/Red (0.7%),3.46,Support,A,-3.47,53.33,15.52,0.49,Blue (0.85%)/Red (0.55%),3.59
254,Zilean,Support,SUPPORT,A,1.68,50.83,81.95,3.38,Blue (0.68%)/Red (0.72%),3.03,Support,A,-2.49,50.95,79.72,2.67,Blue (1.12%)/Red (0.28%),3.11
255,Zoe,Mage,MID,A,4.62,49.84,94.18,3.62,Blue (0.02%)/Red (1.39%),2.58,Mage,A,2.43,49.14,87.85,3.45,Blue (0.27%)/Red (1.18%),2.60
256,Zoe,NaN,SUPPORT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mage,D,-3.55,47.73,10.03,0.37,Blue (0.93%)/Red (0.52%),2.14


In [11]:
for col in percentage_column:
    df_full[col] = df_full[col].astype(float)

In [ ]:
import re
ban_columns = ["Ban % (season 12)", "Ban % (season 13)"]

def get_ban_percentage(x: str):
    if pd.isna(x):
        return x
    
    numbers = re.findall